# To run this script, you will need your user token (generate one if you don't have it).
https://github.com/settings/tokens

And save it as a file called _github_token.txt_ in a folder called _reusable_code_ on the Jupyter home

In [14]:
import github
import requests
import re

In [15]:
# Pick up Github token for authentication purposes 
mytoken=open('/home/jupyter/reusable_code/github_token.txt',"r").read()

In [16]:
#Create GitHub connection/ instance
g = github.Github(mytoken)

# Get your specific user (update with your username)
myuser=g.get_user()
print(myuser.login)

# Subset to just the ITV repo
ITV_org=g.get_organization('ITV').get_repos()

# Subset further to just a list of BritBox DIG repos according to our naming convention
bbdig_repos=[i for i in ITV_org if i.full_name.replace('ITV/','')[:5]=='bbdig']

# Get the bb_insight team
bbinsightteam=g.get_organization('ITV').get_team([x.id for x in g.get_organization('ITV').get_teams() if x.name=='bb_insight'][0])


### Loop all repos in bbdig_repos that you have admin access to, and add the team if they aren't there already

In [17]:
for r in bbdig_repos: # For each BBDIG repo
    try:
        amICollobarator=False
        for x in r.get_collaborators(): # Loop collaborators
            if x.login==myuser.login: # If I'm a collaborator
                    amICollobarator=True
                    if x.permissions.admin==True: #And I have admin permissions
                        Madechanges=False
                        if bbinsightteam.get_repo_permission(r)==None: #Check if the insight team has been added at all (None='No')
                            bbinsightteam.add_to_repos(r) #Check if the team is not added at all (None='No')
                            Madechanges=True
                            bbinsightteam.set_repo_permission(repo=r,permission='admin') # And make them admin
                            print('Added BB Insight team to {}'.format(r.name))
                        else:
                            if bbinsightteam.get_repo_permission(r).admin==False: #If team already exists but isn't admin
                                bbinsightteam.set_repo_permission(repo=r,permission='admin') # Make them admin
                                Madechanges=True
                                print('Made BB Insight team admins on {}'.format(r.name))
                        if Madechanges==False:
                            pass
                            #print('I can edit {} but no changes were required, team is already admin'.format(r.name))
                    else:
                        print('No admin rights on {}'.format(r.name))

        if amICollobarator==False:
            print('I''m not a collaborator on {}'.format(r.name))
    except:
        print('Cannot influence {}'.format(r.name))
